In [1]:
import timm
import glob
import time
import logging
import torch
import torch.nn as nn
import torch.nn.parallel
from torch.autograd import Variable
from collections import OrderedDict
from timm.data import Dataset, create_loader, resolve_data_config,  FastCollateMixup, mixup_batch, AugMixDataset
from timm.models import create_model, resume_checkpoint, convert_splitbn_model, apply_test_time_pool
from timm.utils import *
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy, JsdCrossEntropy
from timm.optim import create_optimizer
from timm.scheduler import create_scheduler
from munch import Munch
import yaml
import sys
from resnet_generator import Generator, Critic
from datetime import datetime
import numpy as np
import torch.distributed as dist
import torch.nn.functional as F
from transformer import Attention
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
from crd.criterion import CRDLoss
from dataset.imagenet import get_dataloader_sample
from timm.data import transforms_factory

In [2]:
# from apex import amp
# from apex.parallel import DistributedDataParallel as DDP
# from apex.parallel import convert_syncbn_model
has_apex = False

In [3]:
# GPU Device
gpu_id = '0,1,2'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device " , use_cuda)

GPU device  True


In [4]:
torch.backends.cudnn.benchmark = True

In [5]:
with open('config/save.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
args = Munch(config)
args.prefetcher = not args.no_prefetcher
args.distributed = False
args.device = 'cuda'
args.world_size = 3
args.rank = 0
logging.info('Training with a single process on %d GPUs.' % args.num_gpu)

INFO:root:Training with a single process on 3 GPUs.


In [6]:
if args.distributed:
    args.num_gpu = 1
    args.device = 'cuda:%d' % args.local_rank
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend='nccl', init_method='env://', rank=args.rank, world_size=args.world_size)
    args.world_size = torch.distributed.get_world_size()
    args.rank = torch.distributed.get_rank()
    assert args.rank >= 0

In [7]:
torch.manual_seed(args.seed + args.rank)

In [8]:
# model_ns = timm.create_model('tf_efficientnet_b1_ns', pretrained=True)
# model_ns = model_ns.cuda()
# model_ns = torch.nn.DataParallel(model_ns)
# model_ns = model_ns.eval()

In [9]:
# fc_name = 'module.classifier.'
# teacher_model_weights = {}
# for name, param in model_ns.named_parameters():
#     teacher_model_weights[name] = param.detach()

In [10]:
model_raw = timm.create_model('tf_efficientnet_b1', pretrained=True)
model_raw = model_raw.cuda()
model_raw = torch.nn.DataParallel(model_raw)
if args.resume:
    print('raw load')
    model_raw.load_state_dict(torch.load('raw').state_dict())

In [11]:
model_g = Generator(args, img_size=args.img_size, max_conv_dim=256)
model_g = model_g.cuda()

In [12]:
model_critic = Critic(args)
model_critic = model_critic.cuda()
model_critic = torch.nn.DataParallel(model_critic)
if args.resume:
    print('critic load')
    model_critic.load_state_dict(torch.load('critic').state_dict())

In [13]:
use_amp = False
if has_apex and args.amp:
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
    model_raw =  amp.initialize(model_raw)
#     model_ns = amp.initialize(model_ns)
    use_amp = True
if args.local_rank == 0:
    logging.info('NVIDIA APEX {}. AMP {}.'.format(
        'installed' if has_apex else 'not installed', 'on' if use_amp else 'off'))

INFO:root:NVIDIA APEX not installed. AMP off.


In [14]:
train_dir = '/home/data/imagenet/train'
val_dir = '/home/data/imagenet/val'
data_config = resolve_data_config(vars(args), model=model_g, verbose=args.local_rank == 0)

INFO:root:Data processing configuration for current model + dataset:
INFO:root:	input_size: (3, 240, 240)
INFO:root:	interpolation: bicubic
INFO:root:	mean: (0.485, 0.456, 0.406)
INFO:root:	std: (0.229, 0.224, 0.225)
INFO:root:	crop_pct: 0.882


In [15]:
num_aug_splits = 0

In [16]:
if args.distributed:
    if args.sync_bn:
        assert not args.split_bn
        try:
            if has_apex:
                model = convert_syncbn_model(model)
            else:
                model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
            if args.local_rank == 0:
                logging.info(
                    'Converted model to use Synchronized BatchNorm. WARNING: You may have issues if using '
                    'zero initialized BN layers (enabled by default for ResNets) while sync-bn enabled.')
        except Exception as e:
            logging.error('Failed to enable Synchronized BatchNorm. Install Apex or Torch >= 1.1')
    if has_apex:
        model = DDP(model, delay_allreduce=True)
    else:
        if args.local_rank == 0:
            logging.info("Using torch DistributedDataParallel. Install NVIDIA Apex for Apex DDP.")
        model = DDP(model, device_ids=[args.local_rank])  # can use device str in Torch >= 1.1
    # NOTE: EMA model does not need to be wrapped by DDP

In [17]:
# train_dataset = Dataset(train_dir)
val_dataset = Dataset(val_dir, load_bytes=False, class_map='')

In [18]:
param_count = sum([m.numel() for m in model_g.parameters()])
logging.info('Model created, param count: %d' % (param_count))

INFO:root:Model created, param count: 3662851


In [19]:
collate_fn = None
# if args.prefetcher and args.mixup > 0:
#     assert not num_aug_splits  # collate conflict (need to support deinterleaving in collate mixup)
#     collate_fn = FastCollateMixup(args.mixup, args.smoothing, args.num_classes)

In [20]:
model_raw, test_time_pool = apply_test_time_pool(model_raw, data_config, args)

In [21]:
transform = transforms_factory.transforms_imagenet_eval(img_size=args.img_size, crop_pct=args.crop_pct, interpolation=args.interpolation)

In [22]:
train_loader = get_dataloader_sample(transform, dataset='imagenet', batch_size=args.batch_size, is_sample=True, k=4096)

stage1 finished!
dataset initialized!
num_samples 1281167
num_class 1000


In [23]:
crop_pct = 1.0 if test_time_pool else data_config['crop_pct']
val_loader = create_loader(
    val_dataset,
    input_size=data_config['input_size'],
    batch_size=args.batch_size,
    is_training=False,
    use_prefetcher=args.prefetcher,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    num_workers=args.workers,
    crop_pct=crop_pct,
    pin_memory=args.pin_mem,
    tf_preprocessing=args.tf_preprocessing)

In [24]:
if args.jsd:
    assert num_aug_splits > 1  # JSD only valid with aug splits set
    train_loss_fn = JsdCrossEntropy(num_splits=num_aug_splits, smoothing=args.smoothing)
    validate_loss_fn = nn.CrossEntropyLoss()
# elif args.mixup > 0.:
#     # smoothing is handled with mixup label transform
#     train_loss_fn = SoftTargetCrossEntropy()
    validate_loss_fn = nn.CrossEntropyLoss()
elif args.smoothing:
    train_loss_fn = LabelSmoothingCrossEntropy(smoothing=args.smoothing)
    validate_loss_fn = nn.CrossEntropyLoss()
else:
    train_loss_fn = nn.CrossEntropyLoss()
    validate_loss_fn = train_loss_fn
    
crd_loss_fn = CRDLoss(args).cuda()
critic_loss_fn = nn.BCELoss()
real_label = 1
fake_label = 0

In [25]:
trainable_list = nn.ModuleList([])
trainable_list.append(model_g)
trainable_list.append(model_raw)
trainable_list.append(crd_loss_fn.embed_s)
trainable_list.append(crd_loss_fn.embed_t)

ModuleList(
  (0): Generator(
    (from_rgb): Conv2d(3, 68, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (encode): ModuleList(
      (0): ResBlk(
        (actv): LeakyReLU(negative_slope=0.2)
        (conv1): Conv2d(68, 68, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(68, 136, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm1): InstanceNorm2d(68, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (norm2): InstanceNorm2d(68, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (conv1x1): Conv2d(68, 136, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): ResBlk(
        (actv): LeakyReLU(negative_slope=0.2)
        (conv1): Conv2d(136, 136, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(136, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm1): InstanceNorm2d(136, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  

In [26]:
optimizer = create_optimizer(args, trainable_list)
lr_scheduler, num_epochs = create_scheduler(args, optimizer)

In [27]:
optimizer_critic = create_optimizer(args, model_critic)
lr_scheduler_critic, _ = create_scheduler(args, optimizer_critic)

In [28]:
# optionally resume from a checkpoint
resume_state = {}
resume_epoch = None
if args.resume:
    resume_state, resume_epoch = resume_checkpoint(model_g, args.resume)
if resume_state and not args.no_resume_opt:
    if 'optimizer' in resume_state:
        if args.local_rank == 0:
            logging.info('Restoring Optimizer state from checkpoint')
        optimizer.load_state_dict(resume_state['optimizer'])
    if use_amp and 'amp' in resume_state and 'load_state_dict' in amp.__dict__:
        if args.local_rank == 0:
            logging.info('Restoring NVIDIA AMP state from checkpoint')
        amp.load_state_dict(resume_state['amp'])
del resume_state

In [29]:
model_g = torch.nn.DataParallel(model_g)

In [30]:
if args.local_rank == 0:
    logging.info('Scheduled epochs: {}'.format(num_epochs))

INFO:root:Scheduled epochs: 200


In [31]:
resume_state = {}
resume_epoch = None
start_epoch = 0
if args.start_epoch is not None:
    # a specified start_epoch will always override the resume epoch
    start_epoch = args.start_epoch
elif resume_epoch is not None:
    start_epoch = resume_epoch
if lr_scheduler is not None and start_epoch > 0:
    lr_scheduler.step(start_epoch)

In [32]:
eval_metric = args.eval_metric
best_metric = None
best_epoch = None
saver = None
output_dir = ''
if args.local_rank == 0:
    output_base = args.output if args.output else './output'
    exp_name = '-'.join([
        datetime.now().strftime("%Y%m%d-%H%M%S"),
        args.model,
        str(data_config['input_size'][-1])
    ])
    output_dir = get_outdir(output_base, 'train', exp_name)
    decreasing = True if eval_metric == 'loss' else False
    saver = CheckpointSaver(checkpoint_dir=output_dir, decreasing=decreasing)

In [33]:
model_ema = None
if args.model_ema:
    # Important to create EMA model after cuda(), DP wrapper, and AMP but before SyncBN and DDP wrapper
    model_ema = ModelEma(
        model,
        decay=args.model_ema_decay,
        device='cpu' if args.model_ema_force_cpu else '',
        resume=args.resume)

In [34]:
def train_epoch(epoch, model_g, model_raw, model_critic, loader, optimizer, optimizer_critic, loss_fn, crd_loss_fn, critic_loss_fn, args,
               lr_scheduler=None, lr_scheduler_critic=None, saver=None, output_dir='', use_amp=False, model_ema=None):

    batch_time_m = AverageMeter()
    data_time_m = AverageMeter()
    losses_m = AverageMeter()
    losses_crd = AverageMeter()
    losses_ce = AverageMeter()
    losses_kd = AverageMeter()
    losses_critic = AverageMeter()
    losses_g = AverageMeter()
    
    model_g.train()
    model_critic.train()
    model_raw.eval()
    
    end = time.time()
    last_idx = len(loader) - 1
    num_updates = epoch * len(loader)
        
    for batch_idx, data in enumerate(loader):
        inputs, target, index, contrast_idx, out_ns, f_ns = data
        inputs, target, index, contrast_idx = inputs.cuda(), target.cuda(), index.cuda(), contrast_idx.cuda()
        last_batch = batch_idx == last_idx
        data_time_m.update(time.time() - end)
        
        g_out = model_g(inputs)
        
        inputs_z = inputs + g_out
        output, f_raw = model_raw(inputs_z)

        # Critic
        model_critic.train()
        model_critic.zero_grad()
        label = torch.full((f_raw.size(0), ), real_label, device='cuda')
        critic_out = model_critic(f_ns).view(-1)
        loss_critic_ns = args.lambda_critic * critic_loss_fn(critic_out, label)
        loss_critic_ns.backward()
        loss_critic_ns_out = loss_critic_ns.item()
        
        label.fill_(fake_label)
        critic_out = model_critic(f_raw.detach()).view(-1)
        loss_critic_raw = critic_loss_fn(critic_out, label)
        loss_critic_raw.backward()
        loss_critic_raw_out = args.lambda_critic * loss_critic_raw.item()
        loss_critic = loss_critic_ns_out + loss_critic_raw_out
        optimizer_critic.step()
        
        out_ns, f_ns = out_ns.cuda(), f_ns.cuda()
        # KD
        p_s = F.log_softmax(output/args.T, dim=1)
        p_t = F.softmax(out_ns/args.T, dim=1)
        loss_kd = F.kl_div(p_s, p_t, size_average=False) * (args.T ** 2) / output.shape[0]
        
       # CE
        loss_ce = loss_fn(output, target)
        
        # CRD
        loss_crd = crd_loss_fn(f_raw, f_ns, index, contrast_idx)
        
        # G
        model_critic.eval()
        label.fill_(real_label)
        critic_g_out = model_critic(f_raw).view(-1)
        loss_g = critic_loss_fn(critic_g_out, label)
        
        # overall loss
        loss = args.lambda_kd * loss_kd + args.lambda_ce * loss_ce + args.lambda_crd * loss_crd + args.lambda_g * loss_g
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if not args.distributed:
            losses_kd.update(loss_kd.item(), inputs.size(0))
            losses_ce.update(loss_ce.item(), inputs.size(0))
            losses_crd.update(loss_crd.item(), inputs.size(0))
            losses_critic.update(loss_critic, inputs.size(0))
            losses_g.update(loss_g.item(), inputs.size(0))
            losses_m.update(loss.item(), inputs.size(0))

        if model_ema is not None:
            model_ema.update(model)
        num_updates += 1

        batch_time_m.update(time.time() - end)
        if last_batch or batch_idx % args.log_interval == 0:
            lrl = [param_group['lr'] for param_group in optimizer.param_groups]
            lr = sum(lrl) / len(lrl)

            if args.distributed:
                reduced_loss = reduce_tensor(loss.data, args.world_size)
                losses_m.update(reduced_loss.item(), inputs.size(0))

            if args.local_rank == 0:
                logging.info(
                    'Train: {} [{:>4d}/{} ({:>3.0f}%)]  '
                    'Loss: {loss.val:>9.6f} ({loss.avg:>6.4f})  '
                    'Loss_kd: {loss_kd.val:>9.6f} ({loss_kd.avg:>6.4f})  '
                    'Loss_ce: {loss_ce.val:>9.6f} ({loss_ce.avg:>6.4f})  '
                    'Loss_crd: {loss_crd.val:>9.6f} ({loss_crd.avg:>6.4f})  '
                    'Loss_g: {loss_g.val:>9.6f} ({loss_g.avg:>6.4f})  '
                    'Loss_critic: {loss_critic.val:>9.6f} ({loss_critic.avg:>6.4f})  '
                    'Time: {batch_time.val:.3f}s, {rate:>7.2f}/s  '
                    '({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'LR: {lr:.7f}  '
                    'Data: {data_time.val:.3f} ({data_time.avg:.3f})'.format(
                        epoch,
                        batch_idx, len(loader),
                        100. * batch_idx / last_idx,
                        loss=losses_m,
                        loss_kd=losses_kd,
                        loss_ce=losses_ce,
                        loss_crd=losses_crd,
                        loss_g=losses_g,
                        loss_critic=losses_critic,
                        batch_time=batch_time_m,
                        rate=inputs.size(0) * args.world_size / batch_time_m.val,
                        rate_avg=inputs.size(0) * args.world_size / batch_time_m.avg,
                        lr=lr,
                        data_time=data_time_m))

                if args.save_images and output_dir:
                    torchvision.utils.save_image(
                        inputs_z,
                        os.path.join(output_dir, 'train-batch-%d.jpg' % batch_idx),
                        padding=0,
                        normalize=True)

        if saver is not None and args.recovery_interval and (
                last_batch or (batch_idx + 1) % args.recovery_interval == 0):
            saver.save_recovery(
                model_raw, optimizer, args, epoch, model_ema=model_ema, use_amp=use_amp, batch_idx=batch_idx)
            
        if lr_scheduler is not None:
            lr_scheduler.step_update(num_updates=num_updates, metric=losses_m.avg)
            lr_scheduler_critic.step_update(num_updates=num_updates, metric=losses_critic.avg)

        end = time.time()
        # end for

    if hasattr(optimizer, 'sync_lookahead'):
        optimizer.sync_lookahead()
        

    return OrderedDict([('loss', losses_m.avg)])

In [35]:
def val_epoch(model_raw, model, val_loader, criterion, args):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model_raw.eval()
    model.eval()
#     model_att.eval()
    with torch.no_grad():
        # warmup, reduce variability of first batch time, especially for comparing torchscript vs non
        end = time.time()
        for i, (inputs, target) in enumerate(val_loader):
            if args.no_prefetcher:
                target = target.cuda()
                inputs = inputs.cuda()
            
            out = model(inputs)
            # synthesizing input + generator
            inputs_out = inputs + out
            # compute output
            output, _ = model_raw(inputs_out)
#             output = model_att(inputs_out, output)
            loss = criterion(output, target)
            
            # measure accuracy and record loss
            acc1, acc5 = accuracy(output.data, target, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(acc1.item(), inputs.size(0))
            top5.update(acc5.item(), inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.log_freq == 0:
                logging.info(
                    'Test: [{0:>4d}/{1}]  '
                    'Time: {batch_time.val:.3f}s ({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'Loss: {loss.val:>7.4f} ({loss.avg:>6.4f})  '
                    'Acc@1: {top1.val:>7.3f} ({top1.avg:>7.3f})  '
                    'Acc@5: {top5.val:>7.3f} ({top5.avg:>7.3f})'.format(
                        i, len(val_loader), batch_time=batch_time,
                        rate_avg=inputs.size(0) / batch_time.avg,
                        loss=losses, top1=top1, top5=top5))

    results = OrderedDict(
        top1=round(top1.avg, 4), top1_err=round(100 - top1.avg, 4),
        top5=round(top5.avg, 4), top5_err=round(100 - top5.avg, 4),
        param_count=round(param_count / 1e6, 2),
        img_size=data_config['input_size'][-1],
        cropt_pct=crop_pct,
        interpolation=data_config['interpolation'])

    logging.info(' * Acc@1 {:.3f} ({:.3f}) Acc@5 {:.3f} ({:.3f})'.format(
       results['top1'], results['top1_err'], results['top5'], results['top5_err']))
    return results

In [ ]:
for epoch in range(start_epoch, num_epochs):
    if args.distributed:
        train_loader.sampler.set_epoch(epoch)

    train_metrics = train_epoch(
        epoch, model_g, model_raw, model_critic, train_loader, optimizer, optimizer_critic, 
        train_loss_fn, crd_loss_fn, critic_loss_fn, args,
        lr_scheduler=lr_scheduler, lr_scheduler_critic=lr_scheduler_critic, 
        saver=saver, output_dir=output_dir,
        use_amp=use_amp, model_ema=model_ema)

    eval_metrics = val_epoch(model_raw, model_g, val_loader, validate_loss_fn, args)

    if model_ema is not None and not args.model_ema_force_cpu:
        if args.distributed and args.dist_bn in ('broadcast', 'reduce'):
            distribute_bn(model_ema, args.world_size, args.dist_bn == 'reduce')

    if lr_scheduler is not None:
        # step LR for next epoch
        lr_scheduler.step(epoch + 1, eval_metrics[eval_metric])
        
    update_summary( 
        epoch, train_metrics, eval_metrics, os.path.join(output_dir, 'summary.csv'),
        write_header=best_metric is None)

    if saver is not None:
    # save proper checkpoint with eval metric
        save_metric = eval_metrics[eval_metric]
        best_metric, best_epoch = saver.save_checkpoint(
            model_g, optimizer, args,
            epoch=epoch, model_ema=model_ema, metric=save_metric, use_amp=use_amp)
        torch.save(model_raw, 'raw')
        torch.save(model_critic, 'critic')
        torch.save(crd_loss_fn.embed_s, 'embed_s')
        torch.save(crd_loss_fn.embed_t, 'embed_t')

/opt/conda/conda-bld/pytorch_1587428398394/work/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.
/home/cutz/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


normalization constant Z_v1 is set to 2830583.0
normalization constant Z_v2 is set to 2833705.5


INFO:root:Train: 0 [   0/8542 (  0%)]  Loss: 19.745628 (19.7456)  Loss_kd:  0.479628 (0.4796)  Loss_ce:  2.300663 (2.3007)  Loss_crd: 20.489492 (20.4895)  Loss_g:  0.717179 (0.7172)  Loss_critic:  1.143565 (1.1436)  Time: 23.121s,   19.46/s  (23.121s,   19.46/s)  LR: 0.0050000  Data: 4.761 (4.761)
INFO:root:Train: 0 [  50/8542 (  1%)]  Loss: 18.713512 (19.0803)  Loss_kd:  0.383708 (0.4019)  Loss_ce:  1.549439 (1.6455)  Loss_crd: 20.176559 (20.5059)  Loss_g:  0.798895 (0.7852)  Loss_critic:  1.119294 (1.1330)  Time: 1.132s,  397.38/s  (1.598s,  281.67/s)  LR: 0.0050000  Data: 0.015 (0.108)
INFO:root:Train: 0 [ 100/8542 (  1%)]  Loss: 18.882088 (18.9973)  Loss_kd:  0.351507 (0.3834)  Loss_ce:  1.466338 (1.5913)  Loss_crd: 20.556026 (20.4948)  Loss_g:  0.774273 (0.7835)  Loss_critic:  1.120955 (1.1272)  Time: 1.119s,  402.18/s  (1.361s,  330.63/s)  LR: 0.0050000  Data: 0.015 (0.062)
INFO:root:Train: 0 [ 150/8542 (  2%)]  Loss: 18.688566 (18.9476)  Loss_kd:  0.363707 (0.3728)  Loss_ce:  1.

INFO:root:Train: 0 [1350/8542 ( 16%)]  Loss: 18.039787 (18.5577)  Loss_kd:  0.372290 (0.3624)  Loss_ce:  1.415281 (1.4981)  Loss_crd: 20.219379 (20.3444)  Loss_g:  0.095889 (0.5271)  Loss_critic:  0.330971 (0.9695)  Time: 1.123s,  400.56/s  (1.141s,  394.50/s)  LR: 0.0050000  Data: 0.015 (0.018)
INFO:root:Train: 0 [1400/8542 ( 16%)]  Loss: 18.133137 (18.5419)  Loss_kd:  0.376391 (0.3631)  Loss_ce:  1.479887 (1.4985)  Loss_crd: 20.244600 (20.3381)  Loss_g:  0.101471 (0.5123)  Loss_critic:  0.388250 (0.9481)  Time: 1.127s,  399.33/s  (1.140s,  394.72/s)  LR: 0.0050000  Data: 0.015 (0.018)
INFO:root:Train: 0 [1450/8542 ( 17%)]  Loss: 18.051277 (18.5258)  Loss_kd:  0.374410 (0.3635)  Loss_ce:  1.480807 (1.4986)  Loss_crd: 20.113174 (20.3316)  Loss_g:  0.131900 (0.4981)  Loss_critic:  0.305467 (0.9265)  Time: 1.111s,  404.87/s  (1.139s,  394.93/s)  LR: 0.0050000  Data: 0.014 (0.018)
INFO:root:Train: 0 [1500/8542 ( 18%)]  Loss: 17.895643 (18.5102)  Loss_kd:  0.367876 (0.3638)  Loss_ce:  1.52

INFO:root:Train: 0 [2750/8542 ( 32%)]  Loss: 17.464043 (18.2213)  Loss_kd:  0.335975 (0.3570)  Loss_ce:  1.421771 (1.4933)  Loss_crd: 19.601711 (20.1746)  Loss_g:  0.031161 (0.2892)  Loss_critic:  0.033834 (0.5310)  Time: 1.128s,  398.83/s  (1.132s,  397.61/s)  LR: 0.0050000  Data: 0.014 (0.016)
INFO:root:Train: 0 [2800/8542 ( 33%)]  Loss: 17.450327 (18.2124)  Loss_kd:  0.323842 (0.3566)  Loss_ce:  1.435117 (1.4931)  Loss_crd: 19.575724 (20.1688)  Loss_g:  0.038485 (0.2847)  Loss_critic:  0.020687 (0.5220)  Time: 1.121s,  401.40/s  (1.132s,  397.69/s)  LR: 0.0050000  Data: 0.015 (0.016)
INFO:root:Train: 0 [2850/8542 ( 33%)]  Loss: 17.506378 (18.2036)  Loss_kd:  0.340267 (0.3561)  Loss_ce:  1.523405 (1.4928)  Loss_crd: 19.492550 (20.1629)  Loss_g:  0.060832 (0.2803)  Loss_critic:  0.026544 (0.5133)  Time: 1.115s,  403.43/s  (1.131s,  397.75/s)  LR: 0.0050000  Data: 0.015 (0.016)
INFO:root:Train: 0 [2900/8542 ( 34%)]  Loss: 17.482662 (18.1946)  Loss_kd:  0.353079 (0.3557)  Loss_ce:  1.41

...

INFO:root:Train: 6 [7300/8542 ( 85%)]  Loss:  3.121759 (3.2211)  Loss_kd:  0.208932 (0.2109)  Loss_ce:  1.363302 (1.3805)  Loss_crd:  1.935934 (2.0347)  Loss_g:  0.000972 (0.0023)  Loss_critic:  0.000271 (0.0014)  Time: 1.113s,  404.32/s  (1.123s,  400.87/s)  LR: 0.0047045  Data: 0.015 (0.015)
INFO:root:Train: 6 [7350/8542 ( 86%)]  Loss:  3.217473 (3.2210)  Loss_kd:  0.208317 (0.2109)  Loss_ce:  1.416111 (1.3806)  Loss_crd:  1.988100 (2.0346)  Loss_g:  0.003207 (0.0023)  Loss_critic:  0.000970 (0.0014)  Time: 1.138s,  395.28/s  (1.123s,  400.87/s)  LR: 0.0047045  Data: 0.015 (0.015)
INFO:root:Train: 6 [7400/8542 ( 87%)]  Loss:  3.159974 (3.2210)  Loss_kd:  0.206398 (0.2109)  Loss_ce:  1.356193 (1.3807)  Loss_crd:  1.995956 (2.0345)  Loss_g:  0.000772 (0.0023)  Loss_critic:  0.000403 (0.0014)  Time: 1.124s,  400.34/s  (1.123s,  400.88/s)  LR: 0.0047045  Data: 0.014 (0.015)
INFO:root:Train: 6 [7450/8542 ( 87%)]  Loss:  3.201286 (3.2210)  Loss_kd:  0.206390 (0.2109)  Loss_ce:  1.372794 (1

INFO:root:Test: [  40/334]  Time: 0.325s (0.502s,  298.80/s)  Loss:  1.3916 (1.4191)  Acc@1:  79.333 ( 83.041)  Acc@5:  96.000 ( 96.081)
INFO:root:Test: [  50/334]  Time: 0.326s (0.468s,  320.77/s)  Loss:  1.3024 (1.3871)  Acc@1:  78.000 ( 84.222)  Acc@5:  98.000 ( 96.366)
INFO:root:Test: [  60/334]  Time: 0.329s (0.445s,  337.08/s)  Loss:  1.2339 (1.3733)  Acc@1:  81.333 ( 83.432)  Acc@5:  95.333 ( 96.306)
INFO:root:Test: [  70/334]  Time: 0.326s (0.428s,  350.17/s)  Loss:  1.3854 (1.3553)  Acc@1:  80.000 ( 82.836)  Acc@5:  97.333 ( 96.366)
INFO:root:Test: [  80/334]  Time: 0.326s (0.416s,  360.67/s)  Loss:  1.5537 (1.3438)  Acc@1:  60.667 ( 82.420)  Acc@5:  98.667 ( 96.387)
INFO:root:Test: [  90/334]  Time: 0.336s (0.406s,  369.21/s)  Loss:  1.8047 (1.3275)  Acc@1:  71.333 ( 82.381)  Acc@5:  91.333 ( 96.484)
INFO:root:Test: [ 100/334]  Time: 0.326s (0.398s,  376.49/s)  Loss:  1.4796 (1.3276)  Acc@1:  81.333 ( 82.429)  Acc@5:  98.667 ( 96.548)
INFO:root:Test: [ 110/334]  Time: 0.326s 

INFO:root:Train: 7 [ 600/8542 (  7%)]  Loss:  3.277108 (3.2128)  Loss_kd:  0.220453 (0.2167)  Loss_ce:  1.438890 (1.3872)  Loss_crd:  2.021124 (2.0013)  Loss_g:  0.001083 (0.0098)  Loss_critic:  0.002001 (0.0089)  Time: 1.116s,  403.16/s  (1.130s,  398.27/s)  LR: 0.0047045  Data: 0.014 (0.022)
INFO:root:Train: 7 [ 650/8542 (  8%)]  Loss:  3.054444 (3.2074)  Loss_kd:  0.211997 (0.2161)  Loss_ce:  1.292652 (1.3847)  Loss_crd:  1.934915 (1.9990)  Loss_g:  0.002328 (0.0093)  Loss_critic:  0.001735 (0.0084)  Time: 1.115s,  403.46/s  (1.129s,  398.67/s)  LR: 0.0047045  Data: 0.015 (0.021)
INFO:root:Train: 7 [ 700/8542 (  8%)]  Loss:  3.221589 (3.2025)  Loss_kd:  0.226851 (0.2156)  Loss_ce:  1.361337 (1.3822)  Loss_crd:  2.039175 (1.9971)  Loss_g:  0.002576 (0.0088)  Loss_critic:  0.002869 (0.0081)  Time: 1.111s,  405.00/s  (1.128s,  398.95/s)  LR: 0.0047045  Data: 0.015 (0.021)
INFO:root:Train: 7 [ 750/8542 (  9%)]  Loss:  3.141292 (3.1989)  Loss_kd:  0.203925 (0.2152)  Loss_ce:  1.319330 (1

INFO:root:Train: 7 [2000/8542 ( 23%)]  Loss:  3.129732 (3.1668)  Loss_kd:  0.213196 (0.2106)  Loss_ce:  1.350199 (1.3686)  Loss_crd:  1.956188 (1.9801)  Loss_g:  0.001734 (0.0044)  Loss_critic:  0.000778 (0.0038)  Time: 1.117s,  402.96/s  (1.122s,  400.96/s)  LR: 0.0047045  Data: 0.015 (0.017)
INFO:root:Train: 7 [2050/8542 ( 24%)]  Loss:  2.949201 (3.1660)  Loss_kd:  0.206364 (0.2105)  Loss_ce:  1.253519 (1.3684)  Loss_crd:  1.861076 (1.9797)  Loss_g:  0.000572 (0.0043)  Loss_critic:  0.000574 (0.0038)  Time: 1.128s,  398.88/s  (1.122s,  400.98/s)  LR: 0.0047045  Data: 0.014 (0.017)
INFO:root:Train: 7 [2100/8542 ( 25%)]  Loss:  3.052332 (3.1655)  Loss_kd:  0.208888 (0.2104)  Loss_ce:  1.295111 (1.3681)  Loss_crd:  1.929546 (1.9794)  Loss_g:  0.005870 (0.0043)  Loss_critic:  0.000575 (0.0037)  Time: 1.113s,  404.34/s  (1.122s,  400.96/s)  LR: 0.0047045  Data: 0.014 (0.017)
INFO:root:Train: 7 [2150/8542 ( 25%)]  Loss:  3.122919 (3.1653)  Loss_kd:  0.194796 (0.2103)  Loss_ce:  1.343337 (1

INFO:root:Train: 7 [3400/8542 ( 40%)]  Loss:  3.145253 (3.1626)  Loss_kd:  0.200125 (0.2091)  Loss_ce:  1.407947 (1.3694)  Loss_crd:  1.920187 (1.9766)  Loss_g:  0.001288 (0.0036)  Loss_critic:  0.000463 (0.0028)  Time: 1.113s,  404.24/s  (1.122s,  401.17/s)  LR: 0.0047045  Data: 0.014 (0.016)
INFO:root:Train: 7 [3450/8542 ( 40%)]  Loss:  3.261626 (3.1623)  Loss_kd:  0.211095 (0.2091)  Loss_ce:  1.443622 (1.3693)  Loss_crd:  2.006492 (1.9764)  Loss_g:  0.002144 (0.0035)  Loss_critic:  0.000815 (0.0027)  Time: 1.115s,  403.41/s  (1.122s,  401.14/s)  LR: 0.0047045  Data: 0.014 (0.016)
INFO:root:Train: 7 [3500/8542 ( 41%)]  Loss:  3.096113 (3.1622)  Loss_kd:  0.215753 (0.2090)  Loss_ce:  1.337975 (1.3693)  Loss_crd:  1.925151 (1.9763)  Loss_g:  0.002831 (0.0035)  Loss_critic:  0.024552 (0.0027)  Time: 1.118s,  402.57/s  (1.122s,  401.09/s)  LR: 0.0047045  Data: 0.015 (0.016)
INFO:root:Train: 7 [3550/8542 ( 42%)]  Loss:  3.153582 (3.1618)  Loss_kd:  0.204937 (0.2089)  Loss_ce:  1.354232 (1